# Initialization

Test notebook for the C-MAPPS benchmark. Test different MLP architectures. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import NonSequenceTunableModelRegression
import CMAPSAuxFunctions

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


# Define architectures

Define each one of the different architectures to be tested.

In [2]:
#Clear the previous tensorflow graph
K.clear_session()
lambda_regularization = 0.20

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model



# Process Data

In [3]:
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 18
window_stride = 2
max_rul = 134

dHandler_cmaps = CMAPSDataHandler(data_folder, 4, selected_features, max_rul, window_size, window_stride)

init
super init


# Build the model

In [4]:
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]


#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
modelRULSN = RULmodel_SN_5(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = NonSequenceTunableModelRegression('ModelRUL_SN_5', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps, 
                                 data_scaler=min_max_scaler)

# Test on dataset 1

In [5]:
tModel.load_data(verbose=1, cross_validation_ratio=0)
tModel.epochs = 100
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

Loading data for dataset 4 with window_size of 18, stride of 2 and maxRUL of 134. Cros-Validation ratio 0
Loading data from file


# Print Out Result

In [6]:
iteration = 10
file = open("dt4TestArchitectureswithrefactored.csv", "w")
csvfile = csv.writer(file, lineterminator='\n')
for j in range(iteration):
    tModel.train_model(learningRate_scheduler=lrate, verbose=1)
    tModel.evaluate_model(['rhs', 'rmse'], round=2)
    print("scores")

    cScores = tModel.scores
    rmse = math.sqrt(cScores['score_1'])
    rmse2 = cScores['rmse']
    rhs = cScores['rhs']
    time = tModel.train_time

    i = range(1,len(tModel.y_test)+1)


    for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, tModel.y_predicted, tModel.y_predicted_rounded, tModel.y_test):
        file.write('\tEngine {}, RUL {}, Rectified RUL {}, Real RUL {}\n'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))
file.write("%s\n"%cScores)
file.write("RMSE2: "+"%s\n"%rmse2)
file.write("RHS: "+"%s\n"%rhs)
file.write("Time: "+"%s\n"%time)
file.close()

Epoch 1/100
28565/28565 [==============================] - 0s 15us/step - loss: 10228.8632 - mean_squared_error: 10220.7098
Epoch 2/100
28565/28565 [==============================] - 0s 6us/step - loss: 7901.0085 - mean_squared_error: 7888.0496
Epoch 3/100
28565/28565 [=